In [ ]:
!pip install deepface
!pip install tqdm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.2 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=bd12c34b69cc492b500e79f94fdf09bc02a06672ef376d0176bf384607501970
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


In [ ]:
!pip install deepface tqdm


In [1]:
pip install python-dotenv

In [ ]:
python-dotenv


In [ ]:
# BlueSky User Scraper: Age 45+, Depression Filter, Gender, Resumable

import requests
import csv
import time
import io
import numpy as np
from PIL import Image
from deepface import DeepFace
import signal
import sys
import os
import random
from dotenv import load_dotenv

# === CONFIG ===
load_dotenv()

USERNAME = os.getenv("BLUESKY_USERNAME")
APP_PASSWORD = os.getenv("BLUESKY_PASSWORD")
BASE_URL = "https://bsky.social/xrpc"
MAX_USERS = 1000
MIN_AGE = 45
SAVE_FILENAME = "filtered_45plus.csv"
AUTOSAVE_INTERVAL = 20

COMMON_WORDS = [
    "the", "and", "is", "in", "life", "day", "love", "happy", "time",
    "news", "this", "post", "today", "was", "hello", "week", "new",
    "share", "real", "feel", "travel", "career", "music", "fitness"
]

DEPRESSION_KEYWORDS = [
    "diagnosed with depression", "i was diagnosed with depression",
    "psychiatrist diagnosed me", "history of depression", "got diagnosed",
    "mental health diagnosis", "dealing with depression", "depression and anxiety",
    "have been diagnosed", "officially diagnosed"
]

collected = []
seen_users = set()

# === SIGNAL HANDLING ===
def autosave_handler(signum, frame):
    print("\n⚠️ Interrupted. Saving progress...")
    save_to_csv(collected, filename="autosave_filtered_45plus.csv")
    sys.exit(0)

signal.signal(signal.SIGINT, autosave_handler)
signal.signal(signal.SIGTERM, autosave_handler)

# === AUTHENTICATION ===
def authenticate(username, password):
    url = f"{BASE_URL}/com.atproto.server.createSession"
    response = requests.post(url, json={"identifier": username, "password": password})
    response.raise_for_status()
    return response.json()["accessJwt"]

# === DEEPFACE PREDICTION ===
def predict_age_gender(avatar_url):
    try:
        if not avatar_url:
            return None, None
        response = requests.get(avatar_url, timeout=10)
        img = Image.open(io.BytesIO(response.content)).convert("RGB")
        result = DeepFace.analyze(
            img_path=np.array(img),
            actions=["age", "gender"],
            enforce_detection=False,
            detector_backend="retinaface"
        )
        if isinstance(result, list):
            result = result[0]
        age = result.get("age")
        gender = result.get("gender")
        return age, gender
    except Exception as e:
        print(f"DeepFace error: {e}")
        return None, None

# === POST CONTENT FILTER ===
def contains_depression_keywords(text):
    text = text.lower()
    return any(keyword in text for keyword in DEPRESSION_KEYWORDS)

# === FILE IO ===
def save_to_csv(data, filename=SAVE_FILENAME):
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["username", "age", "gender", "text", "avatar_url"])
        writer.writerows(data)
    print(f"✅ Saved {len(data)} entries to {filename}")

def load_existing(filename=SAVE_FILENAME):
    if os.path.exists(filename):
        with open(filename, "r", encoding="utf-8") as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                if row:
                    collected.append(row)
                    seen_users.add(row[0])
        print(f"🔄 Resumed from {filename}, found {len(collected)} users")

# === API CALL ===
def search_posts(access_token, keyword, cursor=None):
    url = f"{BASE_URL}/app.bsky.feed.searchPosts"
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {"q": keyword, "limit": 100}
    if cursor:
        params["cursor"] = cursor
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()

# === MAIN LOOP ===
def main():
    global collected, seen_users
    access_token = authenticate(USERNAME, APP_PASSWORD)
    load_existing()

    print("\n🔍 Searching BlueSky posts for users age 45+, excluding depression keywords...")

    try:
        while len(collected) < MAX_USERS:
            keyword = random.choice(COMMON_WORDS)
            cursor = None
            print(f"\n🔎 Searching keyword: '{keyword}'")

            while len(collected) < MAX_USERS:
                try:
                    time.sleep(random.uniform(1.0, 2.0))
                    result = search_posts(access_token, keyword, cursor)
                    posts = result.get("posts", [])
                    if not posts:
                        break

                    for post in posts:
                        record = post.get("record", {})
                        author = post.get("author", {})
                        username = author.get("handle")
                        avatar_url = author.get("avatar", "")
                        text = record.get("text", "")

                        if username in seen_users or not text:
                            continue

                        if contains_depression_keywords(text):
                            continue

                        age, gender = predict_age_gender(avatar_url)
                        print(f"Checking {username} - Age: {age}")

                        if age is None or age < MIN_AGE:
                            continue

                        gender_str = gender.title() if isinstance(gender, str) else max(gender, key=gender.get)
                        collected.append([username, int(age), gender_str, text[:500], avatar_url])
                        seen_users.add(username)

                        print(f"✅ Collected: {username} (Age: {age}, Gender: {gender_str})")

                        if len(collected) % AUTOSAVE_INTERVAL == 0:
                            save_to_csv(collected, filename="autosave_filtered_45plus.csv")

                        if len(collected) >= MAX_USERS:
                            break

                    cursor = result.get("cursor")
                    if not cursor:
                        break

                except Exception as e:
                    print(f"⚠️ Error: {e}")
                    time.sleep(5)

    except Exception as e:
        print(f"❌ Fatal error: {e}")
        save_to_csv(collected, filename="error_filtered_45plus.csv")
        raise

    save_to_csv(collected)

if __name__ == "__main__":
    main()


25-05-05 14:56:34 - Directory /root/.deepface has been created
25-05-05 14:56:34 - Directory /root/.deepface/weights has been created

🔍 Searching BlueSky posts for users age 45+, excluding depression keywords...

🔎 Searching keyword: 'and'


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5
To: /root/.deepface/weights/retinaface.h5


25-05-05 14:56:42 - retinaface.h5 will be downloaded from the url https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5


100%|██████████| 119M/119M [00:00<00:00, 322MB/s]
Action: age:   0%|          | 0/2 [00:00<?, ?it/s]    Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/age_model_weights.h5
To: /root/.deepface/weights/age_model_weights.h5


25-05-05 14:56:53 - age_model_weights.h5 will be downloaded...


...

Action: gender: 100%|██████████| 2/2 [00:01<00:00,  1.56it/s]


Checking ceencrumpets.bsky.social - Age: 25
⚠️ Error: 400 Client Error: Bad Request for url: https://bsky.social/xrpc/app.bsky.feed.searchPosts?q=and&limit=100&cursor=1000
⚠️ Error: 400 Client Error: Bad Request for url: https://bsky.social/xrpc/app.bsky.feed.searchPosts?q=and&limit=100&cursor=1000
⚠️ Error: 400 Client Error: Bad Request for url: https://bsky.social/xrpc/app.bsky.feed.searchPosts?q=and&limit=100&cursor=1000
⚠️ Error: 400 Client Error: Bad Request for url: https://bsky.social/xrpc/app.bsky.feed.searchPosts?q=and&limit=100&cursor=1000
⚠️ Error: 400 Client Error: Bad Request for url: https://bsky.social/xrpc/app.bsky.feed.searchPosts?q=and&limit=100&cursor=1000
⚠️ Error: 400 Client Error: Bad Request for url: https://bsky.social/xrpc/app.bsky.feed.searchPosts?q=and&limit=100&cursor=1000
⚠️ Error: 400 Client Error: Bad Request for url: https://bsky.social/xrpc/app.bsky.feed.searchPosts?q=and&limit=100&cursor=1000
⚠️ Error: 400 Client Error: Bad Request for url: https://bsk

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.




⚠️ Interrupted. Saving progress...
✅ Saved 49 entries to autosave_filtered_45plus.csv
Traceback (most recent call last):
  File "<ipython-input-3-882dd8971bc7>", line 130, in main
    result = search_posts(access_token, keyword, cursor)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-3-882dd8971bc7>", line 110, in search_posts
    response.raise_for_status()
  File "/usr/local/lib/python3.11/dist-packages/requests/models.py", line 1024, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 400 Client Error: Bad Request for url: https://bsky.social/xrpc/app.bsky.feed.searchPosts?q=and&limit=100&cursor=1000

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-882dd8971bc7>", 

TypeError: object of type 'NoneType' has no len()

In [ ]:
l